In [1]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(".."))
from core.viz import plot_line, create_subplot_grid, plot_bar, plot_statistical_strip
from core.s3 import S3AssetManager

In [2]:
from typing import Iterable, Mapping, Tuple, Dict, Any
import plotly.express as px
import plotly.graph_objs as go

import seaborn as sns
import re
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
import statsmodels.formula.api as smf

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from itertools import product
import warnings
warnings.filterwarnings("ignore")

In [3]:
notebook_name = "comayma_tercer_ensayo_inhimol"
s3 = S3AssetManager(notebook_name=notebook_name)

In [4]:
def to_log10_plus1(x: np.ndarray) -> np.ndarray:
    """Transformación log10(UFC + 1). Clipa en 0 para evitar negativos por error."""
    x = np.asarray(x, dtype=float)
    return np.log10(np.clip(x, a_min=0, a_max=None) + 1.0)

In [5]:


df_raw = s3.read_excel("raw/comayma/ensayos/resultados_ensayo3_oct_comayma.xlsx", sheet_name="M.B. 3da Prueba")
df = pd.melt(
    df_raw,
    id_vars=["Lugar", "Etapa","Fecha muestreo", "Dosis", "Tratamiento", "Muestra"],
    value_vars=["MyL", "Coliformes", "E. coli", "Aerobios mesófilos"],
    var_name="Microorganismo",
    value_name="Resultado"
)

df['concat'] = df['Etapa'].astype(str) + ' ' + df['Fecha muestreo'].astype(str)
df['Resultado'] = pd.to_numeric(df['Resultado'], errors='coerce').fillna(1)
df['Resultado'] = df['Resultado'].replace(0, 1)

df['Resultado_log10'] = np.where(df['Resultado'] > 0, np.log10(df['Resultado']), 0)

df["Resultado"] = pd.to_numeric(df["Resultado"], errors="coerce").fillna(0)
df["resultado_clipped"] = df["Resultado"].clip(lower=1.0)  # 1 UFC como mínimo medible
df["log10_Resultado"] = np.log10(df["resultado_clipped"])
df = df[df["Microorganismo"].isin(['MyL', 'Coliformes', 'Aerobios mesófilos'])]

In [6]:
df

,Lugar,Etapa,Fecha muestreo,Dosis,Tratamiento,Muestra,Microorganismo,Resultado,concat,Resultado_log10,resultado_clipped,log10_Resultado
0,SILO 3 INHIMOLD,10 min aplicado,2025-09-12,1.5,Inhimold 50 CF,Muestra 1,MyL,1300.0,10 min aplicado 2025-09-12,3.113943,1300.0,3.113943
1,SILO 3 INHIMOLD,10 min aplicado,2025-09-12,1.5,Inhimold 50 CF,Muestra 2,MyL,2630.0,10 min aplicado 2025-09-12,3.419956,2630.0,3.419956
2,SILO 3 INHIMOLD,10 min aplicado,2025-09-12,1.5,Inhimold 50 CF,Muestra 3,MyL,2500.0,10 min aplicado 2025-09-12,3.397940,2500.0,3.397940
3,SILO 3 INHIMOLD,10 min aplicado,2025-09-12,1.5,Inhimold 50 CF,Muestra 4,MyL,1700.0,10 min aplicado 2025-09-12,3.230449,1700.0,3.230449
4,SILO 3 INHIMOLD,10 min aplicado,2025-09-12,1.5,Inhimold 50 CF,Muestra 5,MyL,1800.0,10 min aplicado 2025-09-12,3.255273,1800.0,3.255273
...,...,...,...,...,...,...,...,...,...,...,...,...
91,SILO 3 INHIMOLD,Seguimiento,2025-10-10,1.5,Inhimold 50 CF,Muestra 4,Aerobios mesófilos,1400.0,Seguimiento 2025-10-10,3.146128,1400.0,3.146128
92,SILO 5 MYCOCURB,Seguimiento,2025-10-06,0.5,Producto competidor,Muestra 1,Aerobios mesófilos,3600.0,Seguimiento 2025-10-06,3.556303,3600.0,3.556303
93,SILO 5 MYCOCURB,Seguimiento,2025-10-06,0.5,Producto competidor,Muestra 2,Aerobios mesófilos,370.0,Seguimiento 2025-10-06,2.568202,370.0,2.568202
94,SILO 5 MYCOCURB,Seguimiento,2025-10-06,0.5,Producto competidor,Muestra 3,Aerobios mesófilos,620.0,Seguimiento 2025-10-06,2.792392,620.0,2.792392


In [7]:


# ==============
resumen_por_etapa = (
    df.groupby(['Tratamiento', 'Microorganismo', 'Etapa'])['Resultado']
      .agg(['mean', 'std', 'count'])
      .reset_index()
)
resumen_por_etapa

,Tratamiento,Microorganismo,Etapa,mean,std,count
0,Inhimold 50 CF,Aerobios mesófilos,10 min aplicado,2760.000000,826.438140,5
1,Inhimold 50 CF,Aerobios mesófilos,Recepción mp,3500.000000,624.499800,3
2,Inhimold 50 CF,Aerobios mesófilos,Seguimiento,682.500000,481.412851,4
3,Inhimold 50 CF,Coliformes,10 min aplicado,1.000000,0.000000,5
4,Inhimold 50 CF,Coliformes,Recepción mp,1.000000,0.000000,3
5,Inhimold 50 CF,Coliformes,Seguimiento,1.000000,0.000000,4
6,Inhimold 50 CF,MyL,10 min aplicado,1986.000000,562.565552,5
7,Inhimold 50 CF,MyL,Recepción mp,1800.000000,608.276253,3
8,Inhimold 50 CF,MyL,Seguimiento,240.000000,179.257729,4
9,Producto competidor,Aerobios mesófilos,10 min aplicado,3000.000000,1172.603940,5


In [8]:
# Pivot ancho para reducciones (vía medias)
mat = resumen_por_etapa.pivot_table(
    index=['Tratamiento', 'Microorganismo'],
    columns='Etapa',
    values=['mean', 'std', 'count']
)
mat.columns = ['_'.join(col) for col in mat.columns.values]
mat

count_10 min aplicado  \
Tratamiento         Microorganismo                              
Inhimold 50 CF      Aerobios mesófilos                    5.0   
                    Coliformes                            5.0   
                    MyL                                   5.0   
Producto competidor Aerobios mesófilos                    5.0   
                    Coliformes                            5.0   
                    MyL                                   5.0   

                                        count_Recepción mp  count_Seguimiento  \
Tratamiento         Microorganismo                                              
Inhimold 50 CF      Aerobios mesófilos                 3.0                4.0   
                    Coliformes                         3.0                4.0   
                    MyL                                3.0                4.0   
Producto competidor Aerobios mesófilos                 3.0                4.0   
                    Coliformes                         3.0                4.0   
                    MyL                                3.0                4.0   

                                        mean_10 min aplicado  \
Tratamiento         Microorganismo                             
Inhimold 50 CF      Aerobios mesófilos                2760.0   
                    Coliformes                           1.0   
                    MyL                               1986.0   
Producto competidor Aerobios mesófilos                3000.0   
                    Coliformes                           1.0   
                    MyL                               2360.0   

                                        mean_Recepción mp  mean_Seguimiento  \
Tratamiento         Microorganismo                                            
Inhimold 50 CF      Aerobios mesófilos        3500.000000             682.5   
                    Coliformes                   1.000000               1.0   
                    MyL                       1800.000000             240.0   
Producto competidor Aerobios mesófilos        3233.333333            1255.0   
                    Coliformes                   1.000000               1.0   
                    MyL                       1933.333333            1195.0   

                                        std_10 min aplicado  std_Recepción mp  \
Tratamiento         Microorganismo                                              
Inhimold 50 CF      Aerobios mesófilos           826.438140        624.499800   
                    Coliformes                     0.000000          0.000000   
                    MyL                          562.565552        608.276253   
Producto competidor Aerobios mesófilos          1172.603940        838.649708   
                    Coliformes                     0.000000          0.000000   
                    MyL                          870.631954        378.593890   

                                        std_Seguimiento  
Tratamiento         Microorganismo                       
Inhimold 50 CF      Aerobios mesófilos       481.412851  
                    Coliformes                 0.000000  
                    MyL                      179.257729  
Producto competidor Aerobios mesófilos      1566.960965  
                    Coliformes                 0.000000  
                    MyL                     1544.894818

In [9]:
# Reducciones log con medias (interpretación estándar en micro: log-reduction)
def safe_log10(x):
    x = np.asarray(x, dtype=float)
    x = np.clip(x, 0, None)  # seguridad
    return np.log10(x)

In [10]:
for etapa in ["Recepción mp","10 min aplicado","Seguimiento"]:
    if f"mean_{etapa}" not in mat.columns:
        # si falta alguna etapa, crear NaN
        mat[f"mean_{etapa}"] = np.nan

mat["Reduccion_Log_10_min"] = safe_log10(mat["mean_Recepción mp"]) - safe_log10(mat["mean_10 min aplicado"])
mat["Reduccion_Log_Seguimiento"] = safe_log10(mat["mean_Recepción mp"]) - safe_log10(mat["mean_Seguimiento"])
mat = mat.reset_index()
mat

,Tratamiento,Microorganismo,count_10 min aplicado,count_Recepción mp,count_Seguimiento,mean_10 min aplicado,mean_Recepción mp,mean_Seguimiento,std_10 min aplicado,std_Recepción mp,std_Seguimiento,Reduccion_Log_10_min,Reduccion_Log_Seguimiento
0,Inhimold 50 CF,Aerobios mesófilos,5.0,3.0,4.0,2760.0,3500.000000,682.5,826.438140,624.499800,481.412851,0.103159,0.709965
1,Inhimold 50 CF,Coliformes,5.0,3.0,4.0,1.0,1.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,Inhimold 50 CF,MyL,5.0,3.0,4.0,1986.0,1800.000000,240.0,562.565552,608.276253,179.257729,-0.042707,0.875061
3,Producto competidor,Aerobios mesófilos,5.0,3.0,4.0,3000.0,3233.333333,1255.0,1172.603940,838.649708,1566.960965,0.032529,0.411007
4,Producto competidor,Coliformes,5.0,3.0,4.0,1.0,1.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
5,Producto competidor,MyL,5.0,3.0,4.0,2360.0,1933.333333,1195.0,870.631954,378.593890,1544.894818,-0.086605,0.208939


In [11]:


def plot_reduccion_tratamientos_plotly(
    df: pd.DataFrame,
    metric: str = "10min",            # "10min" o "seguimiento"
    prefer: str = "Inhimold 50 CF",   # tratamiento para ordenar por impacto
    color_map: dict | None = None,    # {"Inhimold 50 CF":"#1C8074", "Producto competidor":"#94AF92"}
    output_dir: str | None = None,    # p.ej., ROOT_IMAGEN
    filename: str | None = None,
    save_html: bool = True,
    save_png: bool = False,           # requiere kaleido para exportar PNG
    width: int = 1200,
    height: int = 620,
):
    """
    Devuelve: fig (plotly.Figure), bar_df (DataFrame usado), paths (dict con rutas guardadas si aplica).
    """
    assert metric in {"10min", "seguimiento"}, "metric debe ser '10min' o 'seguimiento'"

    def safe_log10(x):
        x = np.asarray(x, dtype=float)
        x = np.clip(x, 1e-9, None)
        return np.log10(x)

    # --- Resumen por etapa ---
    resumen = (
        df.groupby(["Tratamiento", "Microorganismo", "Etapa"])["Resultado"]
          .agg(["mean", "std", "count"])
          .reset_index()
    )

    mat = resumen.pivot_table(
        index=["Tratamiento", "Microorganismo"],
        columns="Etapa",
        values=["mean", "std", "count"]
    )
    mat.columns = ["_".join(col) for col in mat.columns.values]
    mat = mat.reset_index()

    # --- Reducciones (ambas) ---
    # Recepción → 10 min
    mat["Reduccion_Log_10_min"] = (
        safe_log10(mat.get("mean_Recepción mp", np.nan)) -
        safe_log10(mat.get("mean_10 min aplicado", np.nan))
    )
    # Recepción → Seguimiento
    mat["Reduccion_Log_Seguimiento"] = (
        safe_log10(mat.get("mean_Recepción mp", np.nan)) -
        safe_log10(mat.get("mean_Seguimiento", np.nan))
    )

    metric_col = "Reduccion_Log_10_min" if metric == "10min" else "Reduccion_Log_Seguimiento"
    y_title = f"Reducción log10 (Recepción → {'10 min' if metric=='10min' else 'Seguimiento'})"
    titulo = "Reducción inmediata" if metric == "10min" else "Reducción residual"
    title = f"{titulo} en log10 por microorganismo — Comparativo de tratamientos"

    bar_df = mat[["Tratamiento", "Microorganismo", metric_col]].dropna().copy()
    bar_df.rename(columns={metric_col: "Reduccion"}, inplace=True)

    # --- Orden profesional por impacto del tratamiento preferido ---
    if prefer in bar_df["Tratamiento"].unique():
        order_series = (
            bar_df[bar_df["Tratamiento"] == prefer]
            .set_index("Microorganismo")["Reduccion"]
            .sort_values(ascending=False)
        )
        micro_order = list(order_series.index)
    else:
        micro_order = sorted(bar_df["Microorganismo"].unique())

    # Orden de tratamientos (primero prefer y competidor si existe)
    trat_order = [prefer]
    if "Producto competidor" in bar_df["Tratamiento"].unique() and "Producto competidor" != prefer:
        trat_order.append("Producto competidor")
    # agrega cualquier otro tratamiento que exista
    trat_order += [t for t in bar_df["Tratamiento"].unique() if t not in trat_order]

    # --- Colores corporativos por defecto ---
    if color_map is None:
        color_map = {
            "Inhimold 50 CF": "#1C8074",
            "Producto competidor": "#666666",
        }

    # --- Construcción de figura ---
    fig = go.Figure()
    for tr in trat_order:
        sub = bar_df[bar_df["Tratamiento"] == tr].set_index("Microorganismo").reindex(micro_order)
        y_vals = sub["Reduccion"].values
        # % de reducción equivalente: 1 - 10^(-L)
        perc_red = (1 - np.power(10.0, -np.clip(y_vals, a_min=-10, a_max=10))) * 100.0

        fig.add_trace(go.Bar(
            x=micro_order,
            y=y_vals,
            name=tr,
            marker=dict(color=color_map.get(tr, "#4c78a8"), line=dict(width=0)),  # sin bordes
            text=[f"{v:.2f}" if np.isfinite(v) else "" for v in y_vals],
            textposition="outside",
            cliponaxis=False,
            customdata=np.stack([y_vals, perc_red], axis=-1),
            hovertemplate=(
                "<b>%{x}</b><br>"
                f"Tratamiento: <b>{tr}</b><br>"
                "Reducción log10: %{customdata[0]:.3f}<br>"
                "Reducción equivalente: %{customdata[1]:.1f}%<extra></extra>"
            )
        ))

    # --- Estilo profesional ---
    fig.update_layout(
        title={"text": title, "x": 0.02, "xanchor": "left"},
        paper_bgcolor="white",
        plot_bgcolor="white",
        font=dict(family="Arial", size=15, color="black"),
        barmode="group",
        bargroupgap=0.12,
        bargap=0.18,
        legend=dict(
            title="Tratamiento",
            orientation="h",
            yanchor="bottom", y=1.02,
            xanchor="left", x=0.02,
            bgcolor="rgba(255,255,255,0)",
            borderwidth=0
        ),
        margin=dict(l=70, r=40, t=80, b=90),
        height=height,
        width=width
    )
    fig.update_xaxes(
        title="Microorganismo",
        showline=True, linewidth=2, linecolor="black",
        ticks="outside", tickcolor="black", tickwidth=1.5,
        gridcolor="rgba(0,0,0,0.08)"
    )
    fig.update_yaxes(
        title=y_title,
        zeroline=False,
        showline=True, linewidth=2, linecolor="black",
        ticks="outside", tickcolor="black", tickwidth=1.5,
        gridcolor="rgba(0,0,0,0.08)"
    )
    fig.add_hline(y=0, line_width=2, line_color="black")
    fig.show()

    # --- Guardado opcional ---
    paths = {}
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        if filename is None:
            filename = f"barras_reduccion_{'10min' if metric=='10min' else 'seguimiento'}"
        html_path = os.path.join(output_dir, f"{filename}.html")
        if save_html:
            fig.write_html(html_path, include_plotlyjs="cdn", full_html=True)
            paths["html"] = html_path
        if save_png:
            try:
                png_path = os.path.join(output_dir, f"{filename}.png")
                fig.write_image(png_path, scale=3, width=width, height=height)  # requiere kaleido
                paths["png"] = png_path
            except Exception:
                pass

    return fig, bar_df, paths


In [12]:
df

,Lugar,Etapa,Fecha muestreo,Dosis,Tratamiento,Muestra,Microorganismo,Resultado,concat,Resultado_log10,resultado_clipped,log10_Resultado
0,SILO 3 INHIMOLD,10 min aplicado,2025-09-12,1.5,Inhimold 50 CF,Muestra 1,MyL,1300.0,10 min aplicado 2025-09-12,3.113943,1300.0,3.113943
1,SILO 3 INHIMOLD,10 min aplicado,2025-09-12,1.5,Inhimold 50 CF,Muestra 2,MyL,2630.0,10 min aplicado 2025-09-12,3.419956,2630.0,3.419956
2,SILO 3 INHIMOLD,10 min aplicado,2025-09-12,1.5,Inhimold 50 CF,Muestra 3,MyL,2500.0,10 min aplicado 2025-09-12,3.397940,2500.0,3.397940
3,SILO 3 INHIMOLD,10 min aplicado,2025-09-12,1.5,Inhimold 50 CF,Muestra 4,MyL,1700.0,10 min aplicado 2025-09-12,3.230449,1700.0,3.230449
4,SILO 3 INHIMOLD,10 min aplicado,2025-09-12,1.5,Inhimold 50 CF,Muestra 5,MyL,1800.0,10 min aplicado 2025-09-12,3.255273,1800.0,3.255273
...,...,...,...,...,...,...,...,...,...,...,...,...
91,SILO 3 INHIMOLD,Seguimiento,2025-10-10,1.5,Inhimold 50 CF,Muestra 4,Aerobios mesófilos,1400.0,Seguimiento 2025-10-10,3.146128,1400.0,3.146128
92,SILO 5 MYCOCURB,Seguimiento,2025-10-06,0.5,Producto competidor,Muestra 1,Aerobios mesófilos,3600.0,Seguimiento 2025-10-06,3.556303,3600.0,3.556303
93,SILO 5 MYCOCURB,Seguimiento,2025-10-06,0.5,Producto competidor,Muestra 2,Aerobios mesófilos,370.0,Seguimiento 2025-10-06,2.568202,370.0,2.568202
94,SILO 5 MYCOCURB,Seguimiento,2025-10-06,0.5,Producto competidor,Muestra 3,Aerobios mesófilos,620.0,Seguimiento 2025-10-06,2.792392,620.0,2.792392


In [13]:
# 1) Reducción inmediata (Recepción → 10 min)
fig1, data1, paths1 = plot_reduccion_tratamientos_plotly(
    df,
    metric="10min",
    save_html=True,
    save_png=False
)

s3.save_plotly_html(fig1, "barras_reduccion_10min.html")

In [14]:
# 2) Reducción residual (Recepción → Seguimiento)
fig2, data2, paths2 = plot_reduccion_tratamientos_plotly(
    df,
    metric="seguimiento",
    save_html=True,
    save_png=False
)
s3.save_plotly_html(fig2, "barras_reduccion_seguimiento.html")

In [15]:


COLOR_MAP = {
    "Inhimold 50 CF": "#1C8074",
    "Producto competidor": "#666666",
}
ETAPAS_ORDER = ["Recepción mp", "10 min aplicado", "Seguimiento"]
ETIQUETAS = {"Recepción mp": "Recepción", "10 min aplicado": "10 min", "Seguimiento": "Seguimiento"}

def plot_half_violins_dual_trat_fixed_layout(
    df: pd.DataFrame,
    tratamiento_left: str,
    tratamiento_right: str,
    filename: str | None = None
) -> str:
    sub = df[df["Tratamiento"].isin([tratamiento_left, tratamiento_right])].copy()
    sub = sub[sub["Etapa"].isin(ETAPAS_ORDER)]
    sub["Etapa_label"] = sub["Etapa"].map(ETIQUETAS)
    sub["Etapa_label"] = pd.Categorical(sub["Etapa_label"], categories=[ETIQUETAS[e] for e in ETAPAS_ORDER], ordered=True)

    fig = go.Figure()
    first_left = True
    first_right = True

    for etapa_raw in ETAPAS_ORDER:
        etapa_lbl = ETIQUETAS[etapa_raw]

        # Left (negative side)
        vals_left = sub.loc[(sub["Etapa"] == etapa_raw) & (sub["Tratamiento"] == tratamiento_left), "log10_Resultado"].astype(float)
        if not vals_left.empty:
            fig.add_trace(go.Violin(
                x=[etapa_lbl]*len(vals_left),
                y=vals_left,
                name=tratamiento_left,
                legendgroup=tratamiento_left,
                showlegend=first_left,
                side="negative",
                width=0.9,
                points=False,
                scalemode="width",
                meanline_visible=False,
                box_visible=False,
                line=dict(width=1.2, color=COLOR_MAP.get(tratamiento_left, "#1C8074")),
                fillcolor=COLOR_MAP.get(tratamiento_left, "#1C8074"),
                opacity=0.8
            ))
            first_left = False

        # Right (positive side)
        vals_right = sub.loc[(sub["Etapa"] == etapa_raw) & (sub["Tratamiento"] == tratamiento_right), "log10_Resultado"].astype(float)
        if not vals_right.empty:
            fig.add_trace(go.Violin(
                x=[etapa_lbl]*len(vals_right),
                y=vals_right,
                name=tratamiento_right,
                legendgroup=tratamiento_right,
                showlegend=first_right,
                side="positive",
                width=0.9,
                points=False,
                scalemode="width",
                meanline_visible=False,
                box_visible=False,
                line=dict(width=1.2, color=COLOR_MAP.get(tratamiento_right, "#94AF92")),
                fillcolor=COLOR_MAP.get(tratamiento_right, "#94AF92"),
                opacity=0.8
            ))
            first_right = False

    # Medianas por etapa como puntos
    for tr_name, color in [(tratamiento_left, COLOR_MAP.get(tratamiento_left, "#1C8074")),
                           (tratamiento_right, COLOR_MAP.get(tratamiento_right, "#94AF92"))]:
        med = (
            sub[sub["Tratamiento"] == tr_name]
            .groupby("Etapa_label", observed=True)["log10_Resultado"]
            .mean()
            .reindex([ETIQUETAS[e] for e in ETAPAS_ORDER])
        )
        fig.add_trace(go.Scatter(
            x=med.index.tolist(),
            y=med.values.astype(float).tolist(),
            mode="markers",
            name=f"Promedio — {tr_name}",
            legendgroup=tr_name,
            marker=dict(size=9, color=color, line=dict(width=0))
        ))

    # ---- Layout: título multilínea + leyenda abajo + márgenes amplios ----
    title_text = (
        "Distribución de las observaciones por etapa<br>"
        f"<sup>{tratamiento_left} (izq) vs {tratamiento_right} (der)</sup>"
    )

    fig.update_layout(
        title=dict(
            text=title_text,
            x=0.02, xanchor="left",
            y=0.92, yanchor="top",
            font=dict(size=20),
            pad=dict(t=6, b=6, l=0, r=0)
        ),
        paper_bgcolor="white", plot_bgcolor="white",
        font=dict(family="Arial", size=15, color="black"),
        margin=dict(l=70, r=40, t=110, b=110),   # más espacio arriba/abajo
        height=500, width=1100,
        legend=dict(
            orientation="h",
            yanchor="top", y=-0.12,   # leyenda debajo del gráfico
            xanchor="left", x=0.02,
            bgcolor="rgba(255,255,255,0)",
            borderwidth=0
        )
    )
    fig.update_xaxes(
        title="Etapa",
        showline=True, linewidth=2, linecolor="black",
        ticks="outside", tickcolor="black", tickwidth=1.5,
        categoryorder="array", categoryarray=[ETIQUETAS[e] for e in ETAPAS_ORDER],
        gridcolor="rgba(0,0,0,0.08)"
    )
    fig.update_yaxes(
        title="log10(UFC)",
        showline=True, linewidth=2, linecolor="black",
        ticks="outside", tickcolor="black", tickwidth=1.5,
        gridcolor="rgba(0,0,0,0.08)"
    )
    fig.add_annotation(
    x=0.02, y=1.12, xref="paper", yref="paper",
    text="Entre Recepción y Seguimiento: Inhimold 50 CF ≈ 29.5 × menos (−96%); competidor ≈ 4.46 × menos (−77%)",
    showarrow=False, align="left",
    font=dict(size=20, color="black"))
    fig.show()
    return fig

# Ejecutar con el par del dataset
treats = list(df["Tratamiento"].dropna().unique())
new_path = None
f = plot_half_violins_dual_trat_fixed_layout(df, treats[0], treats[1])

s3.save_plotly_html(f, "half_violins_fixed_Inhimold_50_CF_vs_Producto_competidor.html")

